In [1]:
import pandas as pd
import numpy as np
import os
import os.path as osp
import sys
sys.path.append('../../scripts')
import featurizer
sys.path.append('../../../../../scripts/preprocessing')
import preprocessing
import torch
from tqdm.notebook import tqdm
import warnings
from torch import nn
from typing import Union,Optional
from torch import Tensor
from torch.utils.data import DataLoader
from typing import List

In [2]:
df_btc = preprocessing.load_df('../../../../../../data/Binance/BTC_USDT.csv')

warnings.filterwarnings('ignore')

path_btc = '../../../../../../data/data_new/raw/Binance/BTC_USDT/BTC_USDT.csv'

df_btc_feat = featurizer.load_and_featurize(path_btc)

Total NaN : 

timestamp    0
open         0
high         0
low          0
close        0
volume       0
returns      1
dtype: int64

Total NaN : 

timestamp    0
open         0
high         0
low          0
close        0
volume       0
returns      1
dtype: int64



  0%|          | 0/1811 [00:00<?, ?it/s]

We make use of 90.44726670347875% of the data


0it [00:00, ?it/s]

In [3]:
path_eth = '../../../../../../data/data_new/raw/Binance/ETH_USDT/ETH_USDT.csv'

df_eth_feat = featurizer.load_and_featurize(path_eth)

Total NaN : 

timestamp    0
open         0
high         0
low          0
close        0
volume       0
returns      1
dtype: int64



  0%|          | 0/1811 [00:00<?, ?it/s]

We make use of 90.44726670347875% of the data


0it [00:00, ?it/s]

In [4]:
from torch.utils.data import Dataset

class twoday_basic_dset(Dataset):
    
    def __init__(self,df):
        self.X = torch.tensor(df.X.tolist(),dtype = torch.float)
        self.y = torch.tensor(df.y.astype(int).tolist(),dtype = torch.float)
        
    def __getitem__(self,
                    index: int):
        return (self.X[index],self.y[index])
    
    def __len__(self):
        return len(self.X)

In [5]:
class MLP(nn.Module):
    def __init__(
        self,
        embed_dims: Union[int, list],
        num_layers: Optional[int] = None,
        activation: str = "relu",
    ) -> None:
        super().__init__()

        if isinstance(embed_dims, int):
            assert num_layers is not None
            embed_dims = [embed_dims] * num_layers
        else:
            assert num_layers is None
            num_layers = len(embed_dims)

        assert activation == "relu"

        self.embed_dims = embed_dims
        self.num_layers = num_layers
        self.activation = activation

        self.layers = nn.ModuleList(
            [
                sublayer
                for i, (m, n) in enumerate(zip(embed_dims, embed_dims[1:]))
                for sublayer in [nn.Linear(m, n), nn.BatchNorm1d(n), nn.ReLU()]
                if not (i == num_layers - 2 and isinstance(sublayer, nn.ReLU))
            ]
        )


    def forward(self, x: Tensor) -> Tensor:
        for layer in self.layers:
            x = layer(x)
        return x

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [70]:
#import pytorch_lightning as pl
import lightning.pytorch as pl
from lightning.pytorch.callbacks import TQDMProgressBar
#from pytorch_lightning import LightningModule, Trainer
#from pytorch_lightning.callbacks.progress import TQDMProgressBar
from lightning.pytorch.loggers import CSVLogger
#from pytorch_lightning.loggers import CSVLogger
#import torchmetrics.functional.classification as tm
import torchmetrics as tm
from lightning.pytorch.callbacks import Callback
from lightning.pytorch.callbacks.early_stopping import EarlyStopping




class MLP_binary_classifier(pl.LightningModule):
    
    def __init__(self,
                 embed_dims: List[int],
                 lr = 0.001):
        super().__init__()
        self.mlp = MLP(embed_dims)
        self.Classifier_head = nn.Linear(embed_dims[-1],1)
        self.s = nn.Sigmoid()
        self.step_outputs = {"train": [],"val":[],"test":[]}
        self.average_precision = tm.AveragePrecision(task = 'binary')
        self.auroc = {name : tm.AUROC(task = 'binary') for name in ['train','val']}
        self.lr = lr
        
    def forward(self,
                batch):
        
        x,y = batch
        x = self.mlp(x)
        x = self.Classifier_head(x)
        x = self.s(x)
        
        return x
          
    def _step(self,
              name: str,
              batch: List[torch.tensor]):
        
        target = batch[1].to(device)
        predicted_proba = self.forward(batch).flatten()
        criterion = nn.BCELoss()
        loss = criterion(predicted_proba,target)
        self.step_outputs[name].append({"loss": loss, "predicted_proba": predicted_proba, "target": batch[1]})
        
        self.log(
            f"{name}/loss", loss, prog_bar = True, on_epoch=True, logger = True, on_step= (name=="train"), batch_size=batch[1].shape[0]
        )
        
        #if name == 'val':
        #    self.log("val/average_precision",self.average_precision,on_epoch = True,on_step = False)
        
        
        return {"loss": loss, "predicted_proba": predicted_proba, "target": batch[1]}
    
    def training_step(self, batch, batch_idx = None):
        return self._step("train", batch)
    
    def validation_step(self, batch, batch_idx = None):
        return self._step("val", batch)
    
    #def _epoch_end(self, name):
    #    y = torch.hstack([output["target"] for output in self.step_outputs[name]])
    #    y = y.type(torch.long)
    #    print(y)
    #    preds = torch.hstack([output["predicted_proba"] for output in self.step_outputs[name]])
    #    print(preds) 
    
        #self.log(f"{name}/auroc", self.auroc[name])
        #self.log(f"{name}/average_precision", tm.binary_average_precision(preds, y))

        #precision, recall, _ = tm.binary_precision_recall_curve(preds, y)
        #for ten_x in range(1,11):
        #    x = ten_x / 10
        #    if compute_p_at_r(precision, recall, x) != None:
        #        self.log(f"{name}/p@r={x}", compute_p_at_r(precision, recall, x))
#
        #    y_pred = (preds > x).long()
        #    self.log(f"{name}/accuracy@thh={x}", tm.binary_accuracy(y_pred, y))
        #    self.log(f"{name}/recall@thh={x}", tm.binary_recall(y_pred, y))
        #    self.log(f"{name}/precision@thh={x}", tm.binary_precision(y_pred, y))
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer  
    
    #def on_training_epoch_end(self):
    #    outputs = self.step_outputs['train']
    #    return self._epoch_end("train")
    #
    #def on_validation_epoch_end(self):
    #    outputs = self.step_outputs['val']
    #    return self._epoch_end("val")
    
    #def train_dataloader(self):
    #    dloader = dd.get_dataloader(self.batch_size,self.trainset,self.esm_feat,is_inference = False)
    #    return dloader
    #
    #def val_dataloader(self):
    #    dloader = dd.get_dataloader(self.batch_size,self.valset,self.esm_feat,is_inference = True)
    #    return dloader
            

In [64]:
#class MyCallback(pl.Callback):
#    
#    def on_x_epoch_end(self, name, pl_module):
#        # do something with all training_step outputs, for example:
#        epoch_mean = torch.stack(pl_module.step_outputs[name]).mean()
#        pl_module.log(f"{name}_epoch_mean", epoch_mean)
#        # free up the memory
#        pl_module.step_outputs[name].clear()
#        
#    def on_train_epoch_end(self, trainer, pl_module):
#        return self.on_x_epoch_end("train", pl_module)
#        
#    def on_validation_epoch_end(self, trainer, pl_module):
#        return self.on_x_epoch_end("val", pl_module)

In [65]:
def get_dloader(df,
                batch_size,
                num_workers = 64,
                shuffle = True):
    
    dset = twoday_basic_dset(df)
    dloader = DataLoader(dset,batch_size = batch_size,num_workers = num_workers, shuffle = shuffle)
    
    return dloader

In [72]:
train_df = df_btc_feat[:int(0.6*len(df_btc_feat))]
val_df = df_btc_feat[int(0.62*len(df_btc_feat)):int(0.75*len(df_btc_feat))]
test_df = df_btc_feat[int(0.75*len(df_btc_feat)):]

train_loader = get_dloader(train_df,16,64,True)
val_loader =  get_dloader(val_df,256,64,True)
test_df = get_dloader(test_df,256,64,True)

In [73]:
model = MLP_binary_classifier([124,64,16])

In [74]:
early_stop_callback = EarlyStopping(monitor="val/average_precision", min_delta=0.01, patience=5, verbose=False, mode="max")



In [ ]:
trainer = pl.Trainer(accelerator="gpu") #,callbacks = [early_stop_callback])
trainer.fit(model,train_loader,val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type                   | Params
-------------------------------------------------------------
0 | mlp               | MLP                    | 9.2 K 
1 | Classifier_head   | Linear                 | 17    
2 | s                 | Sigmoid                | 0     
3 | average_precision | BinaryAveragePrecision | 0     
-------------------------------------------------------------
9.2 K     Trainable params
0         Non-trainable params
9.2 K     Total params
0.037     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [19]:
pd.read_csv('./lightning_logs/version_27/metrics.csv')

,train/loss_step,epoch,step,val/loss,val/auroc,val/average_precision,train/loss_epoch
0,0.626697,0,49,NaN,NaN,NaN,NaN
1,NaN,0,61,0.509367,0.522436,0.822361,NaN
2,NaN,0,61,NaN,NaN,NaN,0.667748
3,0.607575,1,99,NaN,NaN,NaN,NaN
4,NaN,1,123,0.512355,0.529964,0.825397,NaN
...,...,...,...,...,...,...,...
203,NaN,62,3905,NaN,NaN,NaN,0.420888
204,0.418630,63,3949,NaN,NaN,NaN,NaN
205,NaN,63,3967,1.809072,0.514660,0.828295,NaN
206,NaN,63,3967,NaN,NaN,NaN,0.420627


In [ ]:
metrics = pd.read_csv('./lightning_logs/version_16/metrics.csv')